In [175]:
#import necessary library
import pandas as pd, numpy as np, re, unicodedata, hashlib, string
from IPython.display import HTML
import base64

In [176]:
#import dataset
path = "https://raw.githubusercontent.com/the-buike/healthcare-flu-insights/refs/heads/main/dataset/conditionss.csv"

In [177]:
#Load the data
# We try to parse the 'Date' column as real dates.
conditions = pd.read_csv(path, low_memory=False)

In [178]:
conditions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3780 entries, 0 to 3779
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   START        3780 non-null   object 
 1   STOP         2792 non-null   object 
 2   PATIENT      3780 non-null   object 
 3   ENCOUNTER    3780 non-null   object 
 4   CODE         3780 non-null   float64
 5   DESCRIPTION  3780 non-null   object 
dtypes: float64(1), object(5)
memory usage: 177.3+ KB


In [179]:
#fix date issue
conditions['START'] = pd.to_datetime(conditions['START'],  errors='coerce')
#fix date issue
conditions['STOP'] = pd.to_datetime(conditions['STOP'], errors='coerce')

In [180]:
conditions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3780 entries, 0 to 3779
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   START        3780 non-null   datetime64[ns]
 1   STOP         2792 non-null   datetime64[ns]
 2   PATIENT      3780 non-null   object        
 3   ENCOUNTER    3780 non-null   object        
 4   CODE         3780 non-null   float64       
 5   DESCRIPTION  3780 non-null   object        
dtypes: datetime64[ns](2), float64(1), object(3)
memory usage: 177.3+ KB


In [181]:
#missing values
def missing_summary(df: pd.DataFrame) -> dict:
    """High-level missing data summary."""
    total_cells = df.shape[0] * df.shape[1]
    total_missing = int(df.isna().sum().sum())
    return {
        "rows": df.shape[0],
        "cols": df.shape[1],
        "total_cells": total_cells,
        "total_missing": total_missing,
        "pct_missing_overall": round(100 * total_missing / total_cells, 2) if total_cells else 0.0,
        "rows_with_any_missing": int(df.isna().any(axis=1).sum()),
        "cols_with_any_missing": int(df.isna().any(axis=0).sum()),
        "cols_all_missing": df.columns[df.isna().all()].tolist()
    }

def missing_report(df: pd.DataFrame, sort_by: str = "pct", descending: bool = True) -> pd.DataFrame:
    """
    Per-column missing report.
    sort_by: "pct" or "count"
    """
    m_count = df.isna().sum()
    m_pct = (m_count / len(df) * 100).round(2) if len(df) else 0
    rep = pd.DataFrame({
        "dtype": df.dtypes.astype(str),
        "non_nulls": df.notna().sum(),
        "missing_count": m_count,
        "missing_pct": m_pct
    })
    key = "missing_pct" if sort_by == "pct" else "missing_count"
    return rep.sort_values(key, ascending=not descending)

def missing_flags(df: pd.DataFrame, high_thresh: float = 50.0) -> dict:
    """Quick flags for columns above a threshold of missingness."""
    pct = (df.isna().sum() / len(df) * 100) if len(df) else pd.Series(0, index=df.columns)
    return {
        "high_missing_cols": pct[pct >= high_thresh].sort_values(ascending=False).round(2).to_dict(),
        "low_missing_cols": pct[(pct > 0) & (pct < high_thresh)].sort_values(ascending=False).round(2).to_dict(),
        "no_missing_cols": pct[pct == 0].index.tolist()
    }

In [182]:
# Overall view
summary = missing_summary(conditions)
summary

{'rows': 3780,
 'cols': 6,
 'total_cells': 22680,
 'total_missing': 988,
 'pct_missing_overall': 4.36,
 'rows_with_any_missing': 988,
 'cols_with_any_missing': 1,
 'cols_all_missing': []}

In [183]:
# Column-level table
report = missing_report(conditions, sort_by="pct")
report.head(20)

,dtype,non_nulls,missing_count,missing_pct
STOP,datetime64[ns],2792,988,26.14
START,datetime64[ns],3780,0,0.00
PATIENT,object,3780,0,0.00
ENCOUNTER,object,3780,0,0.00
CODE,float64,3780,0,0.00
DESCRIPTION,object,3780,0,0.00


In [184]:
conditions.head(20)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2020-09-19,2020-09-19,2f20854f-e037-87c8-aabf-10ef6d3b58fe,e8a79576-b8b6-e1cb-0798-d40e3aac6721,314529007.0,Medication review due (situation)
1,2017-10-10,2017-10-10,184668ad-08d8-2c05-cb16-c7040f00b848,35df0e39-8c7b-77b6-d454-d718a7ac3338,314529007.0,Medication review due (situation)
2,2020-10-24,2020-10-24,2f20854f-e037-87c8-aabf-10ef6d3b58fe,23b462f3-35a3-9687-799e-e9ce2254b361,314529007.0,Medication review due (situation)
3,2020-12-26,2020-12-26,2f20854f-e037-87c8-aabf-10ef6d3b58fe,f1aa4eb6-8976-70f0-2fcb-b53c1ebd0952,314529007.0,Medication review due (situation)
4,2017-11-14,2019-09-17,184668ad-08d8-2c05-cb16-c7040f00b848,6d89e990-cbda-5ad8-1859-75c791fb861f,314529007.0,Medication review due (situation)
5,2018-02-07,2018-03-20,184668ad-08d8-2c05-cb16-c7040f00b848,c0749ae8-1a88-eb3b-1dca-3fd8c6c9b1ec,65363002.0,Otitis media (disorder)
6,2021-02-27,2021-02-27,2f20854f-e037-87c8-aabf-10ef6d3b58fe,d4a47078-9bd2-7077-9914-68f28545eee7,314529007.0,Medication review due (situation)
7,2018-05-10,2018-05-20,184668ad-08d8-2c05-cb16-c7040f00b848,a4016143-e6a1-c085-2e96-c66fff0e809a,43878008.0,Streptococcal sore throat (disorder)
8,2018-07-18,2018-09-18,184668ad-08d8-2c05-cb16-c7040f00b848,2016f18e-8965-7805-9552-aa1b5931ad85,65363002.0,Otitis media (disorder)
9,2021-05-29,2024-10-05,2f20854f-e037-87c8-aabf-10ef6d3b58fe,18800c68-b54a-deef-7ef0-bde4e168ed24,314529007.0,Medication review due (situation)


In [185]:
rename_map = {
    "START": "start_date",
    "STOP": "stop_date",
    "PATIENT": "patient",
    "ENCOUNTER": "encounter",
    "CODE": "code",
    "DESCRIPTION": "description",
}
conditions = conditions.rename(columns=rename_map)

In [186]:
# 3) Normalize CODE: remove only a trailing ".0"
def strip_trailing_point_zero(s: pd.Series) -> pd.Series:
    s = s.fillna("").astype(str).str.strip()
    return s.str.replace(r"\.0$", "", regex=True)

conditions["code"] = strip_trailing_point_zero(conditions["code"])

In [187]:
conditions.head(20)

,start_date,stop_date,patient,encounter,code,description
0,2020-09-19,2020-09-19,2f20854f-e037-87c8-aabf-10ef6d3b58fe,e8a79576-b8b6-e1cb-0798-d40e3aac6721,314529007,Medication review due (situation)
1,2017-10-10,2017-10-10,184668ad-08d8-2c05-cb16-c7040f00b848,35df0e39-8c7b-77b6-d454-d718a7ac3338,314529007,Medication review due (situation)
2,2020-10-24,2020-10-24,2f20854f-e037-87c8-aabf-10ef6d3b58fe,23b462f3-35a3-9687-799e-e9ce2254b361,314529007,Medication review due (situation)
3,2020-12-26,2020-12-26,2f20854f-e037-87c8-aabf-10ef6d3b58fe,f1aa4eb6-8976-70f0-2fcb-b53c1ebd0952,314529007,Medication review due (situation)
4,2017-11-14,2019-09-17,184668ad-08d8-2c05-cb16-c7040f00b848,6d89e990-cbda-5ad8-1859-75c791fb861f,314529007,Medication review due (situation)
5,2018-02-07,2018-03-20,184668ad-08d8-2c05-cb16-c7040f00b848,c0749ae8-1a88-eb3b-1dca-3fd8c6c9b1ec,65363002,Otitis media (disorder)
6,2021-02-27,2021-02-27,2f20854f-e037-87c8-aabf-10ef6d3b58fe,d4a47078-9bd2-7077-9914-68f28545eee7,314529007,Medication review due (situation)
7,2018-05-10,2018-05-20,184668ad-08d8-2c05-cb16-c7040f00b848,a4016143-e6a1-c085-2e96-c66fff0e809a,43878008,Streptococcal sore throat (disorder)
8,2018-07-18,2018-09-18,184668ad-08d8-2c05-cb16-c7040f00b848,2016f18e-8965-7805-9552-aa1b5931ad85,65363002,Otitis media (disorder)
9,2021-05-29,2024-10-05,2f20854f-e037-87c8-aabf-10ef6d3b58fe,18800c68-b54a-deef-7ef0-bde4e168ed24,314529007,Medication review due (situation)


In [188]:
date_cols = [c for c in conditions.columns if pd.api.types.is_datetime64_any_dtype(conditions[c])]

for c in date_cols:
    conditions[c] = conditions[c].dt.strftime("%Y-%m-%d")
    conditions[c] = conditions[c].fillna("Unknown")


In [189]:
conditions.head(20)

,start_date,stop_date,patient,encounter,code,description
0,2020-09-19,2020-09-19,2f20854f-e037-87c8-aabf-10ef6d3b58fe,e8a79576-b8b6-e1cb-0798-d40e3aac6721,314529007,Medication review due (situation)
1,2017-10-10,2017-10-10,184668ad-08d8-2c05-cb16-c7040f00b848,35df0e39-8c7b-77b6-d454-d718a7ac3338,314529007,Medication review due (situation)
2,2020-10-24,2020-10-24,2f20854f-e037-87c8-aabf-10ef6d3b58fe,23b462f3-35a3-9687-799e-e9ce2254b361,314529007,Medication review due (situation)
3,2020-12-26,2020-12-26,2f20854f-e037-87c8-aabf-10ef6d3b58fe,f1aa4eb6-8976-70f0-2fcb-b53c1ebd0952,314529007,Medication review due (situation)
4,2017-11-14,2019-09-17,184668ad-08d8-2c05-cb16-c7040f00b848,6d89e990-cbda-5ad8-1859-75c791fb861f,314529007,Medication review due (situation)
5,2018-02-07,2018-03-20,184668ad-08d8-2c05-cb16-c7040f00b848,c0749ae8-1a88-eb3b-1dca-3fd8c6c9b1ec,65363002,Otitis media (disorder)
6,2021-02-27,2021-02-27,2f20854f-e037-87c8-aabf-10ef6d3b58fe,d4a47078-9bd2-7077-9914-68f28545eee7,314529007,Medication review due (situation)
7,2018-05-10,2018-05-20,184668ad-08d8-2c05-cb16-c7040f00b848,a4016143-e6a1-c085-2e96-c66fff0e809a,43878008,Streptococcal sore throat (disorder)
8,2018-07-18,2018-09-18,184668ad-08d8-2c05-cb16-c7040f00b848,2016f18e-8965-7805-9552-aa1b5931ad85,65363002,Otitis media (disorder)
9,2021-05-29,2024-10-05,2f20854f-e037-87c8-aabf-10ef6d3b58fe,18800c68-b54a-deef-7ef0-bde4e168ed24,314529007,Medication review due (situation)


In [190]:
conditions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3780 entries, 0 to 3779
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   start_date   3780 non-null   object
 1   stop_date    3780 non-null   object
 2   patient      3780 non-null   object
 3   encounter    3780 non-null   object
 4   code         3780 non-null   object
 5   description  3780 non-null   object
dtypes: object(6)
memory usage: 177.3+ KB


In [191]:
for col in ["start_date", "stop_date"]:
    # Convert to date, keep NaT for blanks
    conditions[col] = pd.to_datetime(conditions[col], errors="coerce").dt.date

In [192]:
conditions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3780 entries, 0 to 3779
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   start_date   3780 non-null   object
 1   stop_date    2792 non-null   object
 2   patient      3780 non-null   object
 3   encounter    3780 non-null   object
 4   code         3780 non-null   object
 5   description  3780 non-null   object
dtypes: object(6)
memory usage: 177.3+ KB


In [193]:
#fix date issue
conditions['start_date'] = pd.to_datetime(conditions['start_date'],  errors='coerce')
#fix date issue
conditions['stop_date'] = pd.to_datetime(conditions['stop_date'], errors='coerce')

In [194]:
conditions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3780 entries, 0 to 3779
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   start_date   3780 non-null   datetime64[ns]
 1   stop_date    2792 non-null   datetime64[ns]
 2   patient      3780 non-null   object        
 3   encounter    3780 non-null   object        
 4   code         3780 non-null   object        
 5   description  3780 non-null   object        
dtypes: datetime64[ns](2), object(4)
memory usage: 177.3+ KB


In [195]:

def add_viable_flag(
    df: pd.DataFrame,
    essential_cols=None,
    tokens=("","na","n/a","null","none","nan","missing","?","unknown","unk"),
    in_place=True
):
    """
    Marks rows as viable=True/False based on presence of valid values in essential columns.
    - Checks each column for missing tokens (case-insensitive, trims whitespace).
    - Treats actual NaN/None and the tokens above as missing.
    - If essential_cols is None, uses all columns.
    - Returns DataFrame (and adds a 'viable' boolean column).
    """
    if not in_place:
        out = df.copy()
    else:
        out = df

    # Build a boolean mask of "is missing" per cell using tokens
    toks = {str(t).strip().lower() for t in tokens}

    def is_missing_series(s: pd.Series) -> pd.Series:
        # Start with native NA detection
        miss = s.isna()
        # Also catch token-based missing for object-like data
        if s.dtype == "object" or pd.api.types.is_string_dtype(s):
            s_str = s.astype("string").str.strip().str.lower()
            miss = miss | s_str.isin(toks)
        return miss

    # Compute missing mask per column
    miss_map = {c: is_missing_series(out[c]) for c in out.columns}

    # Decide which columns must be valid
    if essential_cols is None:
        essential_cols = list(out.columns)

    # Any non-existent essential column will count as missing
    missing_essential = []
    for c in essential_cols:
        if c in miss_map:
            missing_essential.append(miss_map[c])
        else:
            # Column absent => treat as all missing
            missing_essential.append(pd.Series(True, index=out.index))

    # Row is viable if all essential columns are NOT missing
    if missing_essential:
        any_missing = missing_essential[0].copy()
        for m in missing_essential[1:]:
            any_missing = any_missing | m
        out["viable"] = ~any_missing
    else:
        # If no essential columns specified at all, mark all as True
        out["viable"] = True

    return out


In [196]:
essential = ["start_date", "stop_date", "patient", "encounter", "code", "description"]

conditions = add_viable_flag(conditions, essential_cols=essential, in_place=True)
conditions["viable"].value_counts()

viable
True     2792
False     988
Name: count, dtype: int64

In [197]:
conditions.head(30)

,start_date,stop_date,patient,encounter,code,description,viable
0,2020-09-19,2020-09-19,2f20854f-e037-87c8-aabf-10ef6d3b58fe,e8a79576-b8b6-e1cb-0798-d40e3aac6721,314529007,Medication review due (situation),True
1,2017-10-10,2017-10-10,184668ad-08d8-2c05-cb16-c7040f00b848,35df0e39-8c7b-77b6-d454-d718a7ac3338,314529007,Medication review due (situation),True
2,2020-10-24,2020-10-24,2f20854f-e037-87c8-aabf-10ef6d3b58fe,23b462f3-35a3-9687-799e-e9ce2254b361,314529007,Medication review due (situation),True
3,2020-12-26,2020-12-26,2f20854f-e037-87c8-aabf-10ef6d3b58fe,f1aa4eb6-8976-70f0-2fcb-b53c1ebd0952,314529007,Medication review due (situation),True
4,2017-11-14,2019-09-17,184668ad-08d8-2c05-cb16-c7040f00b848,6d89e990-cbda-5ad8-1859-75c791fb861f,314529007,Medication review due (situation),True
5,2018-02-07,2018-03-20,184668ad-08d8-2c05-cb16-c7040f00b848,c0749ae8-1a88-eb3b-1dca-3fd8c6c9b1ec,65363002,Otitis media (disorder),True
6,2021-02-27,2021-02-27,2f20854f-e037-87c8-aabf-10ef6d3b58fe,d4a47078-9bd2-7077-9914-68f28545eee7,314529007,Medication review due (situation),True
7,2018-05-10,2018-05-20,184668ad-08d8-2c05-cb16-c7040f00b848,a4016143-e6a1-c085-2e96-c66fff0e809a,43878008,Streptococcal sore throat (disorder),True
8,2018-07-18,2018-09-18,184668ad-08d8-2c05-cb16-c7040f00b848,2016f18e-8965-7805-9552-aa1b5931ad85,65363002,Otitis media (disorder),True
9,2021-05-29,2024-10-05,2f20854f-e037-87c8-aabf-10ef6d3b58fe,18800c68-b54a-deef-7ef0-bde4e168ed24,314529007,Medication review due (situation),True


In [198]:
#import dataset
path = "https://raw.githubusercontent.com/the-buike/healthcare-flu-insights/refs/heads/main/dataset/encounter.csv"

In [199]:
#Load the data
# We try to parse the 'Date' column as real dates.
encounter = pd.read_csv(path, low_memory=False)

HTTPError: HTTP Error 404: Not Found

In [200]:
encounter.head(10)

,Id,start,stop,patient,encounter_class,code,description,total_claim_cost,payer_coverage,start_dow
0,35df0e39-8c7b-77b6-d454-d718a7ac3338,2017-10-10,2017-10-10,184668ad-08d8-2c05-cb16-c7040f00b848,wellness,410620009,Well child visit (procedure),931.67,831.67,Tuesday
1,e8a79576-b8b6-e1cb-0798-d40e3aac6721,2020-09-19,2020-09-19,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1192.34,0.00,Saturday
2,23b462f3-35a3-9687-799e-e9ce2254b361,2020-10-24,2020-10-24,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1135.60,695.95,Saturday
3,f1aa4eb6-8976-70f0-2fcb-b53c1ebd0952,2020-12-26,2020-12-26,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1171.70,937.36,Saturday
4,6d89e990-cbda-5ad8-1859-75c791fb861f,2017-11-14,2017-11-14,184668ad-08d8-2c05-cb16-c7040f00b848,wellness,410620009,Well child visit (procedure),272.80,272.80,Tuesday
5,d4a47078-9bd2-7077-9914-68f28545eee7,2021-02-27,2021-02-27,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1149.00,919.20,Saturday
6,c2c9d365-027e-5094-8ad2-88850d83de08,2018-01-16,2018-01-16,184668ad-08d8-2c05-cb16-c7040f00b848,wellness,410620009,Well child visit (procedure),816.80,816.80,Tuesday
7,c0749ae8-1a88-eb3b-1dca-3fd8c6c9b1ec,2018-02-07,2018-02-07,184668ad-08d8-2c05-cb16-c7040f00b848,outpatient,185345009,Encounter for symptom (procedure),85.55,35.55,Wednesday
8,e91516a5-0c90-628d-31a7-e0924e93f814,2018-03-20,2018-03-20,184668ad-08d8-2c05-cb16-c7040f00b848,wellness,410620009,Well child visit (procedure),816.80,816.80,Tuesday
9,18800c68-b54a-deef-7ef0-bde4e168ed24,2021-05-29,2021-05-29,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),816.80,653.44,Saturday


In [201]:
keep_cols = [
    "Id",
    "PATIENT",
    "START",
    "STOP",
    "ENCOUNTERCLASS",
    "CODE",
    "DESCRIPTION",
    "TOTAL_CLAIM_COST",
    "PAYER_COVERAGE"
]

encounter.drop(columns=[c for c in encounter.columns if c not in keep_cols], inplace=True)


In [202]:
encounter.head(20)

,Id
0,35df0e39-8c7b-77b6-d454-d718a7ac3338
1,e8a79576-b8b6-e1cb-0798-d40e3aac6721
2,23b462f3-35a3-9687-799e-e9ce2254b361
3,f1aa4eb6-8976-70f0-2fcb-b53c1ebd0952
4,6d89e990-cbda-5ad8-1859-75c791fb861f
5,d4a47078-9bd2-7077-9914-68f28545eee7
6,c2c9d365-027e-5094-8ad2-88850d83de08
7,c0749ae8-1a88-eb3b-1dca-3fd8c6c9b1ec
8,e91516a5-0c90-628d-31a7-e0924e93f814
9,18800c68-b54a-deef-7ef0-bde4e168ed24


In [ ]:
rename_map = {
    "START": "start",
    "STOP": "stop",
    "PATIENT": "patient",
    "ENCOUNTERCLASS": "encounter_class",
    "CODE": "code",
    "DESCRIPTION": "description",
    "TOTAL_CLAIM_COST": "total_claim_cost",
    "PAYER_COVERAGE": "payer_coverage",
}
encounter = encounter.rename(columns=rename_map)

In [ ]:
encounter.head()

,Id,start,stop,patient,encounter_class,code,description,total_claim_cost,payer_coverage
0,35df0e39-8c7b-77b6-d454-d718a7ac3338,2017-10-10T01:36:56Z,2017-10-10T01:51:56Z,184668ad-08d8-2c05-cb16-c7040f00b848,wellness,410620009,Well child visit (procedure),931.67,831.67
1,e8a79576-b8b6-e1cb-0798-d40e3aac6721,2020-09-19T04:03:59Z,2020-09-19T04:18:59Z,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1192.34,0.00
2,23b462f3-35a3-9687-799e-e9ce2254b361,2020-10-24T04:03:59Z,2020-10-24T04:18:59Z,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1135.60,695.95
3,f1aa4eb6-8976-70f0-2fcb-b53c1ebd0952,2020-12-26T04:03:59Z,2020-12-26T04:18:59Z,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1171.70,937.36
4,6d89e990-cbda-5ad8-1859-75c791fb861f,2017-11-14T01:36:56Z,2017-11-14T01:51:56Z,184668ad-08d8-2c05-cb16-c7040f00b848,wellness,410620009,Well child visit (procedure),272.80,272.80


In [ ]:
encounter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6190 entries, 0 to 6189
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                6190 non-null   object 
 1   start             6190 non-null   object 
 2   stop              6190 non-null   object 
 3   patient           6190 non-null   object 
 4   encounter_class   6190 non-null   object 
 5   code              6190 non-null   int64  
 6   description       6190 non-null   object 
 7   total_claim_cost  6190 non-null   float64
 8   payer_coverage    6190 non-null   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 435.4+ KB


In [ ]:
encounter["start"] = pd.to_datetime(encounter["start"], errors="coerce", utc=True)
encounter["stop"]  = pd.to_datetime(encounter["stop"],  errors="coerce", utc=True)

In [ ]:
encounter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6190 entries, 0 to 6189
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   Id                6190 non-null   object             
 1   start             6190 non-null   datetime64[ns, UTC]
 2   stop              6190 non-null   datetime64[ns, UTC]
 3   patient           6190 non-null   object             
 4   encounter_class   6190 non-null   object             
 5   code              6190 non-null   int64              
 6   description       6190 non-null   object             
 7   total_claim_cost  6190 non-null   float64            
 8   payer_coverage    6190 non-null   float64            
dtypes: datetime64[ns, UTC](2), float64(2), int64(1), object(4)
memory usage: 435.4+ KB


In [ ]:
encounter.head()

,Id,start,stop,patient,encounter_class,code,description,total_claim_cost,payer_coverage
0,35df0e39-8c7b-77b6-d454-d718a7ac3338,2017-10-10 01:36:56+00:00,2017-10-10 01:51:56+00:00,184668ad-08d8-2c05-cb16-c7040f00b848,wellness,410620009,Well child visit (procedure),931.67,831.67
1,e8a79576-b8b6-e1cb-0798-d40e3aac6721,2020-09-19 04:03:59+00:00,2020-09-19 04:18:59+00:00,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1192.34,0.00
2,23b462f3-35a3-9687-799e-e9ce2254b361,2020-10-24 04:03:59+00:00,2020-10-24 04:18:59+00:00,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1135.60,695.95
3,f1aa4eb6-8976-70f0-2fcb-b53c1ebd0952,2020-12-26 04:03:59+00:00,2020-12-26 04:18:59+00:00,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1171.70,937.36
4,6d89e990-cbda-5ad8-1859-75c791fb861f,2017-11-14 01:36:56+00:00,2017-11-14 01:51:56+00:00,184668ad-08d8-2c05-cb16-c7040f00b848,wellness,410620009,Well child visit (procedure),272.80,272.80


In [ ]:
# Parse to timezone-aware datetimes using the renamed columns
encounter["start_dt"] = pd.to_datetime(encounter["start"], errors="coerce", utc=True)
encounter["stop_dt"]  = pd.to_datetime(encounter["stop"],  errors="coerce", utc=True)

In [ ]:
encounter["start_date"] = encounter["start_dt"].dt.date
encounter["stop_date"]  = encounter["stop_dt"].dt.date
encounter["start_dow"]  = encounter["start_dt"].dt.day_name()

In [ ]:
encounter.head(10)

,Id,start,stop,patient,encounter_class,code,description,total_claim_cost,payer_coverage,start_dt,stop_dt,start_date,stop_date,start_dow
0,35df0e39-8c7b-77b6-d454-d718a7ac3338,2017-10-10,2017-10-10,184668ad-08d8-2c05-cb16-c7040f00b848,wellness,410620009,Well child visit (procedure),931.67,831.67,2017-10-10 00:00:00+00:00,2017-10-10 00:00:00+00:00,2017-10-10,2017-10-10,Tuesday
1,e8a79576-b8b6-e1cb-0798-d40e3aac6721,2020-09-19,2020-09-19,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1192.34,0.00,2020-09-19 00:00:00+00:00,2020-09-19 00:00:00+00:00,2020-09-19,2020-09-19,Saturday
2,23b462f3-35a3-9687-799e-e9ce2254b361,2020-10-24,2020-10-24,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1135.60,695.95,2020-10-24 00:00:00+00:00,2020-10-24 00:00:00+00:00,2020-10-24,2020-10-24,Saturday
3,f1aa4eb6-8976-70f0-2fcb-b53c1ebd0952,2020-12-26,2020-12-26,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1171.70,937.36,2020-12-26 00:00:00+00:00,2020-12-26 00:00:00+00:00,2020-12-26,2020-12-26,Saturday
4,6d89e990-cbda-5ad8-1859-75c791fb861f,2017-11-14,2017-11-14,184668ad-08d8-2c05-cb16-c7040f00b848,wellness,410620009,Well child visit (procedure),272.80,272.80,2017-11-14 00:00:00+00:00,2017-11-14 00:00:00+00:00,2017-11-14,2017-11-14,Tuesday
5,d4a47078-9bd2-7077-9914-68f28545eee7,2021-02-27,2021-02-27,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1149.00,919.20,2021-02-27 00:00:00+00:00,2021-02-27 00:00:00+00:00,2021-02-27,2021-02-27,Saturday
6,c2c9d365-027e-5094-8ad2-88850d83de08,2018-01-16,2018-01-16,184668ad-08d8-2c05-cb16-c7040f00b848,wellness,410620009,Well child visit (procedure),816.80,816.80,2018-01-16 00:00:00+00:00,2018-01-16 00:00:00+00:00,2018-01-16,2018-01-16,Tuesday
7,c0749ae8-1a88-eb3b-1dca-3fd8c6c9b1ec,2018-02-07,2018-02-07,184668ad-08d8-2c05-cb16-c7040f00b848,outpatient,185345009,Encounter for symptom (procedure),85.55,35.55,2018-02-07 00:00:00+00:00,2018-02-07 00:00:00+00:00,2018-02-07,2018-02-07,Wednesday
8,e91516a5-0c90-628d-31a7-e0924e93f814,2018-03-20,2018-03-20,184668ad-08d8-2c05-cb16-c7040f00b848,wellness,410620009,Well child visit (procedure),816.80,816.80,2018-03-20 00:00:00+00:00,2018-03-20 00:00:00+00:00,2018-03-20,2018-03-20,Tuesday
9,18800c68-b54a-deef-7ef0-bde4e168ed24,2021-05-29,2021-05-29,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),816.80,653.44,2021-05-29 00:00:00+00:00,2021-05-29 00:00:00+00:00,2021-05-29,2021-05-29,Saturday


In [ ]:
cols_to_drop = [
    "start_dt","stop_dt","start_date","stop_date","start_time","stop_time",
    "start_hour","start_dow","los_hours","los_days"
]
encounter.drop(columns=cols_to_drop, errors="ignore", inplace=True)

In [ ]:
encounter.head()

,Id,start,stop,patient,encounter_class,code,description,total_claim_cost,payer_coverage,start_dt,stop_dt,start_date,stop_date,start_dow
0,35df0e39-8c7b-77b6-d454-d718a7ac3338,2017-10-10,2017-10-10,184668ad-08d8-2c05-cb16-c7040f00b848,wellness,410620009,Well child visit (procedure),931.67,831.67,2017-10-10 00:00:00+00:00,2017-10-10 00:00:00+00:00,2017-10-10,2017-10-10,Tuesday
1,e8a79576-b8b6-e1cb-0798-d40e3aac6721,2020-09-19,2020-09-19,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1192.34,0.00,2020-09-19 00:00:00+00:00,2020-09-19 00:00:00+00:00,2020-09-19,2020-09-19,Saturday
2,23b462f3-35a3-9687-799e-e9ce2254b361,2020-10-24,2020-10-24,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1135.60,695.95,2020-10-24 00:00:00+00:00,2020-10-24 00:00:00+00:00,2020-10-24,2020-10-24,Saturday
3,f1aa4eb6-8976-70f0-2fcb-b53c1ebd0952,2020-12-26,2020-12-26,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1171.70,937.36,2020-12-26 00:00:00+00:00,2020-12-26 00:00:00+00:00,2020-12-26,2020-12-26,Saturday
4,6d89e990-cbda-5ad8-1859-75c791fb861f,2017-11-14,2017-11-14,184668ad-08d8-2c05-cb16-c7040f00b848,wellness,410620009,Well child visit (procedure),272.80,272.80,2017-11-14 00:00:00+00:00,2017-11-14 00:00:00+00:00,2017-11-14,2017-11-14,Tuesday


In [ ]:
encounter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6190 entries, 0 to 6189
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   Id                6190 non-null   object             
 1   start             6190 non-null   object             
 2   stop              6190 non-null   object             
 3   patient           6190 non-null   object             
 4   encounter_class   6190 non-null   object             
 5   code              6190 non-null   int64              
 6   description       6190 non-null   object             
 7   total_claim_cost  6190 non-null   float64            
 8   payer_coverage    6190 non-null   float64            
 9   start_dt          6190 non-null   datetime64[ns, UTC]
 10  stop_dt           6190 non-null   datetime64[ns, UTC]
 11  start_date        6190 non-null   object             
 12  stop_date         6190 non-null   object             
 13  sta

In [ ]:
cols_to_drop = [
    "start_dt","stop_dt","start_date","stop_date"
]
encounter.drop(columns=cols_to_drop, errors="ignore", inplace=True)

In [ ]:
encounter.head()

,Id,start,stop,patient,encounter_class,code,description,total_claim_cost,payer_coverage,start_dow
0,35df0e39-8c7b-77b6-d454-d718a7ac3338,2017-10-10,2017-10-10,184668ad-08d8-2c05-cb16-c7040f00b848,wellness,410620009,Well child visit (procedure),931.67,831.67,Tuesday
1,e8a79576-b8b6-e1cb-0798-d40e3aac6721,2020-09-19,2020-09-19,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1192.34,0.00,Saturday
2,23b462f3-35a3-9687-799e-e9ce2254b361,2020-10-24,2020-10-24,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1135.60,695.95,Saturday
3,f1aa4eb6-8976-70f0-2fcb-b53c1ebd0952,2020-12-26,2020-12-26,2f20854f-e037-87c8-aabf-10ef6d3b58fe,wellness,410620009,Well child visit (procedure),1171.70,937.36,Saturday
4,6d89e990-cbda-5ad8-1859-75c791fb861f,2017-11-14,2017-11-14,184668ad-08d8-2c05-cb16-c7040f00b848,wellness,410620009,Well child visit (procedure),272.80,272.80,Tuesday


In [ ]:
encounter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6190 entries, 0 to 6189
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                6190 non-null   object 
 1   start             6190 non-null   object 
 2   stop              6190 non-null   object 
 3   patient           6190 non-null   object 
 4   encounter_class   6190 non-null   object 
 5   code              6190 non-null   int64  
 6   description       6190 non-null   object 
 7   total_claim_cost  6190 non-null   float64
 8   payer_coverage    6190 non-null   float64
 9   start_dow         6190 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 483.7+ KB


In [ ]:
#fix date issue
encounter['start'] = pd.to_datetime(encounter['start'],  errors='coerce')
#fix date issue
encounter['stop'] = pd.to_datetime(encounter['stop'], errors='coerce')

In [ ]:
encounter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6190 entries, 0 to 6189
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Id                6190 non-null   object        
 1   start             6190 non-null   datetime64[ns]
 2   stop              6190 non-null   datetime64[ns]
 3   patient           6190 non-null   object        
 4   encounter_class   6190 non-null   object        
 5   code              6190 non-null   int64         
 6   description       6190 non-null   object        
 7   total_claim_cost  6190 non-null   float64       
 8   payer_coverage    6190 non-null   float64       
 9   start_dow         6190 non-null   object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(5)
memory usage: 483.7+ KB


In [ ]:
#import dataset
immun_path = "https://raw.githubusercontent.com/the-buike/healthcare-flu-insights/refs/heads/main/dataset/immunizations.csv"

In [ ]:
immun = pd.read_csv(immun_path, low_memory=False)